# PASAR PARAMETROS Y PONER UNA CELDA NEUVA CON LOS VALORES PASADOS COMO PARAMETROS

In [1]:
import os
import papermill as pm
from IPython.display import display, HTML

def procesar_notebooks(notebooks, directorio_notebooks_procesados="./notebooks_procesados"):
    """
    Ejecuta una lista de notebooks de Jupyter con parámetros especificados y
    guarda los resultados en el directorio de notebooks procesados.

    Parámetros:
    notebooks (list): Una lista de tuplas con el nombre del notebook, la ruta de salida y los parámetros a pasar.
    directorio_notebooks_procesados (str): El directorio donde se guardarán los notebooks procesados.
    """
    # Asegurarse de que el directorio de notebooks procesados exista
    if not os.path.exists(directorio_notebooks_procesados):
        os.makedirs(directorio_notebooks_procesados)  # Crea el directorio si no existe
        print(f"Directorio {directorio_notebooks_procesados} creado.")

    # Ejecutar cada notebook con sus parámetros específicos
    for input_path, output_path, params in notebooks:
        print(f"Ejecutando {input_path} con parámetros {params}...")
        pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            parameters=params
        )
        # Crear el hipervínculo
        display(HTML(f"Ejecutado correctamente, verifique el resultado en el siguiente link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
        print()
        
directorio_procesados = "./notebooks_procesados"
# Lista de notebooks con parámetros específicos
notebooks_con_parametros = [
    ("1_sumar.ipynb", f"{directorio_procesados}/sumar_ejecutado.ipynb", {"x": 10, "y": 5}),
    ("2_restar.ipynb", f"{directorio_procesados}/restar_ejecutado.ipynb", {"x": 10, "y": 3}),
    ("3_multiplicar.ipynb",f"{directorio_procesados}/multiplicar_ejecutado.ipynb", {"x": 4, "y": 7}),
    ("4_dividir.ipynb", f"{directorio_procesados}/dividir_ejecutado.ipynb", {"x": 20, "y": 5}),
]

# Llamar a la función para ejecutar los notebooks
procesar_notebooks(notebooks_con_parametros)

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'


Ejecutando 1_sumar.ipynb con parámetros {'x': 10, 'y': 5}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'



Ejecutando 2_restar.ipynb con parámetros {'x': 10, 'y': 3}...


...


Ejecutando 3_multiplicar.ipynb con parámetros {'x': 4, 'y': 7}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'



Ejecutando 4_dividir.ipynb con parámetros {'x': 20, 'y': 5}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]

# PASAR PARAMETROS Y ELIMINAR LA CELDA DONDE SE ESPERABAN LOS PARAMETROS Y ADICIONAR LA CELDA CON LOS PARAMETROS ADICONADOS

In [2]:
import os
import papermill as pm
from IPython.display import display, HTML
import json

def reemplazar_parametros(notebook_path, nuevos_parametros):
    """
    Reemplaza la celda de parámetros en el notebook con los nuevos valores proporcionados.
    Elimina la celda con tag 'parameters' después de reemplazar los valores.

    Parámetros:
    notebook_path (str): Ruta al archivo del notebook.
    nuevos_parametros (dict): Diccionario con los nuevos valores de los parámetros a reemplazar.
    """
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)
    
    # Buscar la celda con el tag 'parameters'
    for cell in notebook['cells']:
        if 'metadata' in cell and 'tags' in cell['metadata'] and 'parameters' in cell['metadata']['tags']:
            # Reemplazar los parámetros en la celda
            cell['source'] = "\n".join([f"{k} = {v}" for k, v in nuevos_parametros.items()])
            
            # Esperar a eliminar la celda después de reemplazar los valores
            break  # Ya hemos encontrado y modificado la celda, no necesitamos continuar buscando
    
    # Guardar el notebook modificado con los parámetros ya reemplazados
    with open(notebook_path, 'w') as f:
        json.dump(notebook, f, indent=4)

    # Ahora eliminamos la celda después de que los valores hayan sido reemplazados
    for cell in notebook['cells']:
        if 'metadata' in cell and 'tags' in cell['metadata'] and 'parameters' in cell['metadata']['tags']:
            notebook['cells'].remove(cell)  # Eliminar la celda después de reemplazar los valores
            break

    # Guardar el notebook modificado (con la celda de parámetros eliminada)
    with open(notebook_path, 'w') as f:
        json.dump(notebook, f, indent=4)

def procesar_notebooks(notebooks, directorio_notebooks_procesados="./notebooks_procesados"):
    """
    Ejecuta una lista de notebooks de Jupyter con parámetros especificados y
    guarda los resultados en el directorio de notebooks procesados.

    Parámetros:
    notebooks (list): Una lista de tuplas con el nombre del notebook, la ruta de salida y los parámetros a pasar.
    directorio_notebooks_procesados (str): El directorio donde se guardarán los notebooks procesados.
    """
    # Asegurarse de que el directorio de notebooks procesados exista
    if not os.path.exists(directorio_notebooks_procesados):
        os.makedirs(directorio_notebooks_procesados)  # Crea el directorio si no existe
        print(f"Directorio {directorio_notebooks_procesados} creado.")

    # Ejecutar cada notebook con sus parámetros específicos
    for input_path, output_path, params in notebooks:
        # Reemplazar los parámetros en el notebook antes de ejecutarlo
        reemplazar_parametros(input_path, params)
        
        # Ejecutar el notebook con los parámetros actualizados
        print(f"Ejecutando {input_path} con parámetros {params}...")
        pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            parameters=params
        )
        
        # Crear el hipervínculo
        display(HTML(f"Ejecutado correctamente, verifique el resultado en el siguiente link -> <a href='{output_path}' target='_blank'>{output_path}</a>"))
        print()

# Directorio donde se almacenarán los notebooks procesados
directorio_procesados = "./notebooks_procesados"

# Lista de notebooks con parámetros específicos
notebooks_con_parametros = [
    ("1_sumar.ipynb", f"{directorio_procesados}/sumar_ejecutado.ipynb", {"x": 10, "y": 5}),
    ("2_restar.ipynb", f"{directorio_procesados}/restar_ejecutado.ipynb", {"x": 10, "y": 3}),
    ("3_multiplicar.ipynb", f"{directorio_procesados}/multiplicar_ejecutado.ipynb", {"x": 4, "y": 7}),
    ("4_dividir.ipynb", f"{directorio_procesados}/dividir_ejecutado.ipynb", {"x": 20, "y": 5}),
]

# Llamar a la función para ejecutar los notebooks
procesar_notebooks(notebooks_con_parametros)

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'


Ejecutando 1_sumar.ipynb con parámetros {'x': 10, 'y': 5}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'



Ejecutando 2_restar.ipynb con parámetros {'x': 10, 'y': 3}...


...


Ejecutando 3_multiplicar.ipynb con parámetros {'x': 4, 'y': 7}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]

Passed unknown parameter: x
Passed unknown parameter: y
Input notebook does not contain a cell with tag 'parameters'



Ejecutando 4_dividir.ipynb con parámetros {'x': 20, 'y': 5}...


Executing:   0%|          | 0/3 [00:00<?, ?cell/s]